# Assignment 3
Luke Schipper  
CS4210 - Machine Learning  
10 April 2021  

My goal for this assignment was to take the NN function from our lectures and make it scalable. The NN function should be able to take in any value for number of layers, hidden units, and outputs. Furthermore, regularization and different activation functions must be supported. To do this, I

* converted forward and back propagation to use any number of hidden layers, rather than just a single hidden layer. 
* changed the last layer L from sigmoid to softmax. L - 1 layers can use either tanh, relu, or sigmoid. 
* modified the cost function to use categorical cross-entropy rather than binary cross-entropy
* added L2 regularization with weight decay in back propagation

Upon completing these tasks, I trained various models in order to maximize accuracy on the MNIST test set. Therefore, no cross-validation set was used to tune parameters. The model that, after training, demonstrated the highest accuracy on the test set was selected.

The highest accuracy achieved was 98.13%. 

In [ ]:
!pip install tensorflow

import numpy as np
import tensorflow as tf
import time # To measure training time.
from tensorflow.keras import datasets, utils

In [2]:
# This function initializes the weights and biases with small random values. It is mostly
# the same as the original code, but has been converted to use a variable number of hidden
# layers.
def init_params(n_x, n_hl, n_hu, n_y):
    params = {}
    np.random.seed(2)
    params['W1'] = np.random.randn(n_hu, n_x) * 0.01
    params['b1'] = np.zeros((n_hu, 1))
    if n_hl > 1:
      for i in range(2, n_hl + 1): 
        params['W' + str(i)] = np.random.randn(n_hu, n_hu) * 0.01
        params['b' + str(i)] = np.zeros((n_hu, 1))
    params['W' + str(n_hl + 1)] = np.random.randn(n_y, n_hu) * 0.01
    params['b' + str(n_hl + 1)] = np.zeros((n_y, 1))
    return params

In [3]:
# The activation functions that can be used for all layers except the last.
# activations['sigmoid'][0] is the function itself (for forward prop)
# activations['sigmoid'][1] is the derivative (for back prop)
activations = {
    'sigmoid': (
        lambda x: 1/(1 + np.exp(-x)), 
        lambda x: 1/(1 + np.exp(-x)) - (1/(1 + np.exp(-x)))**2
    ),
    'tanh': (
        lambda x: np.tanh(x),
        lambda x: 1 - np.tanh(x)**2
    ),
    'relu': (
        lambda x: x * (x > 0),
        lambda x: 1 * (x > 0)
    ),
}

# The softmax activation for the final layer. Gives a valid probability distributation.
# Used http://www.adeveloperdiary.com/data-science/deep-learning/neural-network-with-softmax-in-python/
# as a guide for back prop equation derivation and python implementation.
def softmax(x):
  exps = np.exp(x - np.max(x))
  return exps / exps.sum(axis = 0, keepdims = True)

# Standard forward propagation adapted to use multiple hidden layers.
# Can choose activation used by all layers except the last, which is stictly softmax.
def forward_propagation(X, params, n_hl, act_key):
    forward = {}
    Z1 = np.dot(params['W1'], X) + params['b1']
    forward['Z1'] = Z1
    forward['A1'] = activations[act_key][0](Z1)
    if n_hl > 1:
      for i in range(2, n_hl + 1):
        Z = np.dot(params['W' + str(i)], forward['A' + str(i - 1)]) + params['b' + str(i)]
        forward['Z' + str(i)] = Z
        forward['A' + str(i)] = activations[act_key][0](Z)
    Zlast = np.dot(params['W' + str(n_hl + 1)], forward['A' + str(n_hl)]) + params['b' + str(n_hl + 1)]
    Alast = softmax(Zlast)
    forward['Z' + str(n_hl + 1)] = Zlast
    forward['A' + str(n_hl + 1)] = Alast
    return Alast, forward

In [4]:
# Compute cost to display when training.
def compute_cost(Y_pred, Y, params, n_hl, lmda):
    m = Y.shape[1]
    # Compute average generalized cross-entropy. Add epsilon to avoid np.log(0).
    entropy_cost = -(1/m) * np.sum(Y * np.log(Y_pred + 1e-8))
    # Compute L2 regularization cost from weights.
    reg_cost = 0
    for i in range(1, n_hl + 2):
      reg_cost += np.sum(params['W' + str(i)]**2)
    # Multiply by hyperparameter lambda, average.
    reg_cost *= lmda/(2 * m)
    return entropy_cost + reg_cost

In [5]:
# Back propagation adapted to use variable hidden layers, activations. Can modify lambda
# to increase/reduce regularization effect. 
# Used https://towardsdatascience.com/deriving-the-backpropagation-equations-from-scratch-part-1-343b300c585a
# for generic back propagation equations (for functions other than sigmoid).
# Used https://towardsdatascience.com/understanding-the-scaling-of-l%C2%B2-regularization-in-the-context-of-neural-networks-e3d25f8b50db
# for weight decay equations.
def backward_propagation(X, Y, params, forward, n_hl, lmda, act_key):
    grads = {}
    m = X.shape[1]
    dZlast = forward['A' + str(n_hl + 1)] - Y
    grads['dZ' + str(n_hl + 1)] = dZlast
    # Generic gradient calculations with weight decay added at the end.
    grads['dW' + str(n_hl + 1)] = (1/m) * np.dot(dZlast, forward['A' + str(n_hl)].T) + (lmda/m) * params['W' + str(n_hl + 1)]
    grads['db' + str(n_hl + 1)] = (1/m) * np.sum(dZlast, axis=1, keepdims=True)
    if n_hl > 1:
      for i in range(n_hl, 1, -1):
        dZ = np.multiply(np.dot(params['W' + str(i + 1)].T, grads['dZ' + str(i + 1)]), activations[act_key][1](forward['Z' + str(i)]))
        grads['dZ' + str(i)] = dZ
        grads['dW' + str(i)] = (1/m) * np.dot(dZ, forward['A' + str(i - 1)].T) + (lmda/m) * params['W' + str(i)]
        grads['db' + str(i)] = (1/m) * np.sum(dZ, axis=1, keepdims=True)
    dZ1 = np.multiply(np.dot(params['W2'].T, grads['dZ2']), activations[act_key][1](forward['Z1']))
    grads['dZ1'] = dZ1
    grads['dW1'] = (1/m) * np.dot(dZ1, X.T) + (lmda/m) * params['W1']
    grads['db1'] = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
    return grads

In [6]:
# Helper function to update weights with gradients at the end of back propagation.
# Same as before but adapted for multiple hidden layers.
def update_params(params, grads, n_hl, learn_rate):
    for i in range(1, n_hl + 2):
      params['W' + str(i)] = params['W' + str(i)] - learn_rate * grads['dW' + str(i)]
      params['b' + str(i)] = params['b' + str(i)] - learn_rate * grads['db' + str(i)]
    return params

In [7]:
# The original NN funciton.
# n_hl = number of hidden layers
# n_hu = number of hidden units
# act_key = function to use for L-1 layers (sigmoid, relu, tanh)
def nn_model(X, Y, n_hl, n_hu, lmda, act_key, num_iterations, learn_rate, print_cost = True, print_freq = 0):
    n_x = X.shape[0]
    n_y = Y.shape[0]
    # by default, print 5 times on each training.
    print_freq = print_freq if print_freq > 0 else num_iterations // 5
    params = init_params(n_x, n_hl, n_hu, n_y)
    cost = 0
    for i in range(0, num_iterations):
        A, forward = forward_propagation(X, params, n_hl, act_key)
        cost = compute_cost(A, Y, params, n_hl, lmda)
        grads = backward_propagation(X, Y, params, forward, n_hl, lmda, act_key)
        params = update_params(params, grads, n_hl, learn_rate)
        if print_cost and i % print_freq == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    if print_cost:
        # print final cost at the end of training
        print("Final cost: %f" %(cost))
    return params

In [8]:
# Uses NN parameters from training to predict a set of test examples.
def predict(X, params, n_hl, act_key):
    Y_pred, forward = forward_propagation(X, params, n_hl, act_key)
    # The output is the index of the largest probability in each prediction.
    # ex. [0.05, 0.05, 0.05, 0.05, 0.6, 0.05, 0.05, 0.05, 0.025, 0.025]
    # predicts 4, since its probability is the highest at 60%.
    output = np.argmax(Y_pred, axis=0)
    return output

In [9]:
# Reports accuracy from NN predictions and the correct labels.
def accuracy(Y_pred, Y):
    return 100 * (np.sum(Y_pred == Y) / Y.shape[0])

In [10]:
# Taken from the lecture on ConvNets. Reshapes the MNIST dataset to two dimensions.
# Normalizes gray level values to be between 0 and 1.
# Labels are encoded with one-hot.
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
Xtrain = train_images.reshape(60000, 784).astype('float32').T / 255.0
Xtest = test_images.reshape(10000, 784).astype('float32').T / 255.0
Ytrain = tf.keras.utils.to_categorical(train_labels, 10).T
Ytest = tf.keras.utils.to_categorical(test_labels, 10).T

In [30]:
# Helper function to train models with various combinations of hyperparameter values.
# Reports training cost, training time, accuracy, and the best run at the end.
def parameter_tuning(layer_list, unit_list, rate_list, lmda_list, act_list, num_iterations):
    results = [];
    for n_hl in layer_list:
        for n_hu in unit_list:
            for rate in rate_list:
                for lmda in lmda_list:
                    for act in act_list:
                        current_run = { 
                            'run': len(results) + 1,
                            'n_hl': n_hl,
                            'n_hu': n_hu,
                            'rate': rate,
                            'lmda': lmda,
                            'act': act
                        }
                        print('Run start ------------------------')
                        print('\n'.join("{}:\t{}".format(k, v) for k, v in current_run.items()))
                        start = time.time()
                        params = nn_model(Xtrain, Ytrain, n_hl, n_hu, lmda, act, num_iterations, rate, True)
                        end = time.time()
                        predictions = predict(Xtest, params, n_hl, act)
                        acc = accuracy(predictions, test_labels)
                        current_run['acc'] = acc
                        current_run['params'] = params
                        print(f'Accuracy: {acc}%')
                        print(f'Train time: {(end - start) // 60} minutes')
                        print('Run end --------------------------', end = '\n\n')
                        results.append(current_run)
    best_run = max(results, key = lambda x: x['acc'])
    print('Best run -------------------------')
    print('\n'.join("{}:\t{}".format(k, '{}' if isinstance(v, dict) else v) for k, v in best_run.items()))
    print('Run end --------------------------')
    return best_run, results

After some initial testing, I found that adding more layers did not improve accuracy with a fixed, low number of training iterations. For the network to converge with more layers added, many more training iterations are needed.

Furthermore, increasing the number of hidden units seemed to greatly improve performance. Starting with 4 hidden units on a single hidden layer NN, adding 6 units brought accuracy above 90% after only 1000 iterations. Adding 100 units brought accuracy around ~96%.

After some research, I discovered that, in theory, a NN with 2 hidden layers can model any decision boundary, meaning it is not usually beneficial to add more. 
(see https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw)

Furthermore, the ideal number of hidden units tends to fall between the number of inputs (784) and the number of outputs (10). Therefore, I narrowed my parameter search space to this range.

Due to time constraints, I decided to keep num_iterations fairly low for the initial tuning. The goal here was to narrow the search space for number of layers and hidden units, while also choosing the best activation function.

In [21]:
num_iterations = 1000
layer_list = [1, 2] # Try 1 and 2 layers for reasons mentioned above.
unit_list = [268, 526] # Try 1/3rd increments between 10 and 784.
rate_list = [1] # Keep fixed.
lmda_list = [0.0001] # Keep regularization effect fixed and very low.
act_list = ['tanh', 'sigmoid', 'relu'] # Try all activation functions.
best_run, results = parameter_tuning(layer_list, unit_list, rate_list, lmda_list, act_list, num_iterations)


Run start ------------------------
run:	1
n_hl:	1
n_hu:	268
rate:	1
lmda:	0.0001
act:	tanh
Cost after iteration 0: 2.302403
Cost after iteration 200: 5.504408
Cost after iteration 400: 0.189902
Cost after iteration 600: 0.114033
Cost after iteration 800: 0.082633
Final cost: 0.067865
Accuracy: 96.71%
Train time: 18.0 minutes
Run end --------------------------

Run start ------------------------
run:	2
n_hl:	1
n_hu:	268
rate:	1
lmda:	0.0001
act:	sigmoid
Cost after iteration 0: 2.305423
Cost after iteration 200: 0.348008
Cost after iteration 400: 0.279593
Cost after iteration 600: 0.244734
Cost after iteration 800: 0.216966
Final cost: 0.193975
Accuracy: 94.31%
Train time: 19.0 minutes
Run end --------------------------

Run start ------------------------
run:	3
n_hl:	1
n_hu:	268
rate:	1
lmda:	0.0001
act:	relu
Cost after iteration 0: 2.301998
Cost after iteration 200: 0.167412
Cost after iteration 400: 0.105562
Cost after iteration 600: 0.077154
Cost after iteration 800: 0.059944
Final 

Across the board, n_hl = 1 performed way better than n_hl = 2. RELU performed slightly better than tanh and sigmoid with n_hl = 1, both in accuracy and in training time. Between n_hu = 268 and n_hu = 526, there was a 0.02 difference in accuracy. With more training iterations, n_hu = 526 might do better, but due to time constraints, I favored lower training time.

I then try more tuning to find the best number of hidden units, keeping hidden layers fixed at 1. With less combinations of parameters, I increased num_iterations to give the larger networks a better chance to converge.

In [14]:
num_iterations = 2000 # Increase iterations.
layer_list = [1] # Keep fixed (best performing).
unit_list = [96, 182, 268, 354, 440] # Try 1/6th increments between 10 and 526.
rate_list = [1] # Keep fixed.
lmda_list = [0.0001] # Keep fixed.
act_list = ['relu'] # Keep fixed (best performing).
best_run, results = parameter_tuning(layer_list, unit_list, rate_list, lmda_list, act_list, num_iterations)

Run start ------------------------
run:	1
n_hl:	1
n_hu:	96
rate:	1
lmda:	0.0001
act:	relu
Cost after iteration 0: 2.303917
Cost after iteration 400: 0.118304
Cost after iteration 800: 0.071523
Cost after iteration 1200: 0.050105
Cost after iteration 1600: 0.037255
Final cost: 0.028459
Accuracy: 97.71%
Train time: 14.0 minutes
Run end --------------------------

Run start ------------------------
run:	2
n_hl:	1
n_hu:	182
rate:	1
lmda:	0.0001
act:	relu
Cost after iteration 0: 2.303443
Cost after iteration 400: 0.106396
Cost after iteration 800: 0.060236
Cost after iteration 1200: 0.039389
Cost after iteration 1600: 0.027538
Final cost: 0.020086
Accuracy: 97.88%
Train time: 17.0 minutes
Run end --------------------------

Run start ------------------------
run:	3
n_hl:	1
n_hu:	268
rate:	1
lmda:	0.0001
act:	relu
Cost after iteration 0: 2.301997
Cost after iteration 400: 0.105562
Cost after iteration 800: 0.059944
Cost after iteration 1200: 0.039559
Cost after iteration 1600: 0.027739
Final

The best performing run had n_hu = 440. However, this is a very small accuracy increase from n_hu = 182, which took half the time to train. Because of this, I chose n_hu = 182, n_hl = 1 for my NN architecture.

I tried to tune learning rate and regularization in order to reduce any overfitting that might be occurring. With fast training times on this NN architecture, I also increased num_iterations to see if performance would be improved.

In [29]:
num_iterations = 4000 # Increase iterations.
layer_list = [1] # Keep fixed (best performance).
unit_list = [182] # Keep fixed (best performance).
rate_list = [1, 0.75, 0.5] # Try different learning rates (0.25 increments).
lmda_list = [0.001, 0.01, 0.1] # Try different lambdas (logarithmic increments).
act_list = ['relu'] # Keep fixed (best performing).
best_run, results = parameter_tuning(layer_list, unit_list, rate_list, lmda_list, act_list, num_iterations)


Run start ------------------------
run:	1
n_hl:	1
n_hu:	182
rate:	1
lmda:	0.001
act:	relu
Cost after iteration 0: 2.303443
Cost after iteration 800: 0.060234
Cost after iteration 1600: 0.027539
Cost after iteration 2400: 0.015121
Cost after iteration 3200: 0.009321
Final cost: 0.006277
Accuracy: 98.05%
Train time: 50.0 minutes
Run end --------------------------

Run start ------------------------
run:	2
n_hl:	1
n_hu:	182
rate:	1
lmda:	0.01
act:	relu
Cost after iteration 0: 2.303444
Cost after iteration 800: 0.060258
Cost after iteration 1600: 0.027589
Cost after iteration 2400: 0.015174
Cost after iteration 3200: 0.009377
Final cost: 0.006333
Accuracy: 98.06%
Train time: 50.0 minutes
Run end --------------------------

Run start ------------------------
run:	3
n_hl:	1
n_hu:	182
rate:	1
lmda:	0.1
act:	relu
Cost after iteration 0: 2.303455
Cost after iteration 800: 0.060623
Cost after iteration 1600: 0.027906
Cost after iteration 2400: 0.015594
Cost after iteration 3200: 0.009900
Final 

The best run had slightly improved accuracy in comparison to learning_rate = 1 and lambda = 0.0001. Therefore, I chose lambda = 0.01 and learning_rate = 0.5, along with n_hl = 1 and n_hu = 182, as my final NN architecture.

I tried one last run of my chosen NN architecture with greatly increased training iterations, to see if there was any performance to be gained.

In [31]:
num_iterations = 8000 # Increase iterations.
# All parameters below deemed best perfoming.
layer_list = [1]
unit_list = [182]
rate_list = [0.5]
lmda_list = [0.01]
act_list = ['relu']
best_run, results = parameter_tuning(layer_list, unit_list, rate_list, lmda_list, act_list, num_iterations)

Run start ------------------------
run:	1
n_hl:	1
n_hu:	182
rate:	0.5
lmda:	0.01
act:	relu
Cost after iteration 0: 2.303444
Cost after iteration 1600: 0.059439
Cost after iteration 3200: 0.026180
Cost after iteration 4800: 0.014099
Cost after iteration 6400: 0.008571
Final cost: 0.005754
Accuracy: 98.13%
Train time: 67.0 minutes
Run end --------------------------

Best run -------------------------
run:	1
n_hl:	1
n_hu:	182
rate:	0.5
lmda:	0.01
act:	relu
acc:	98.13
params:	{}
Run end --------------------------


The best accuracy I could achieve was 98.13%, given training time constraints. I believe the best NN architectures manage to get ~99%. In conclusion, a standard FFNN does pretty well with the MNIST dataset.